In [24]:
import pandas as pd
import numpy as np

In [25]:
def seperate_id(col):
    # Extract country names from the column names
    temp_list = raw[col].unique()

    # Create a mapping from country names to unique IDs
    temp_id_map = {temp: idx + 1 for idx, temp in enumerate(sorted(temp_list))}

    # Convert the dictionary to a DataFrame and save it to a CSV file
    temp_df = pd.DataFrame(list(temp_id_map.items()), columns=[col, col + ' ID'])
    temp_df = temp_df[[col + ' ID', col]]
    temp_df.to_csv(col + '.csv', index=False)

In [26]:
def setting_id(temp_df, col, num, col_rank):
    df = temp_df.copy()

    temp = pd.read_csv(col + '.csv')

    # Create the country_id column by mapping the country names
    df[col + '_id'] = df[col].map(temp.set_index(col)[col + ' ID'])

    # Ensure the values are integers before formatting
    df[col + '_id'] = df[col + '_id'].apply(lambda x: f'{x:0{num}d}')

    cols = df.columns.tolist()
    cols.insert(col_rank, cols.pop(cols.index(col + '_id')))
    df = df[cols]
    
    return df

In [27]:
def manual_id(col, dep_col, num, col_rank, df):
    temp = pd.read_excel(col + ".xlsx")
# Merge the resulting DataFrame with category to get Category
    temp_df = df.merge(temp, left_on = dep_col, right_on = dep_col, how='left')

# Rename 'Category' to 'category_id'
    temp_df.rename(columns={col: col + '_id'}, inplace=True)

# Change the category_id into a string and add leading zeros if necessary
    temp_df[col + '_id'] = temp_df[col + '_id'].apply(lambda x: f'{int(x):0{num}d}' if pd.notnull(x) else '00')

    cols = temp_df.columns.tolist()
    cols.insert(col_rank, cols.pop(cols.index(col + '_id')))
    temp_df = temp_df[cols]
    
    return temp_df

In [28]:
# Read the Excel file
raw = pd.read_excel("rawdata.xlsx")

# Delete some of the country
raw = raw[~raw['Country Name'].isin(['Burundi', 
                                     'Cambodia', 
                                     'Central African Republic',
                                     'Honduras',
                                     'Hong Kong SAR, China',
                                     'Kyrgyz Republic',
                                     'Liberia',
                                     'Nicaragua',
                                     'Papua New Guinea',
                                     'Saudi Arabia',
                                     'Somalia',
                                     'Tajikistan',
                                     'United Arab Emirates'])]

# Delete rows where 'Series Name' contains '%'
raw = raw[~raw['Series Name'].str.contains('%')]
raw = raw[~raw['Series Name'].str.contains('female')]
raw = raw[~raw['Series Name'].str.contains('male')]
raw = raw[~raw['Series Name'].str.contains('constant')]

In [29]:
seperate_id('Country Name')
seperate_id('Series Name')

In [30]:
raw_country = setting_id(raw, 'Country Name', 2, 0)
raw_series = setting_id(raw_country, 'Series Name', 3, 0)
raw_category = manual_id('Category', 'Series Name', 2, 2, raw_series)

In [31]:
raw_data = raw_category.copy()
# Add a new column 'data_id' starting from 1 for the first row
raw_data['Data_id'] = range(1, len(raw) + 1)

raw_data['Data_id'] = raw_data['Data_id'].apply(lambda x: f'{int(x):06}' if pd.notnull(x) else '000000')

cols = raw_data.columns.tolist()
cols.insert(3, cols.pop(cols.index('Data_id')))
raw_data = raw_data[cols]

In [32]:
raw_id = raw_data.copy()

# Create the 'id' column by combining 'series_id', 'country_id', and 'data_id'
raw_id['id'] = raw_data['Series Name_id'] + raw_data['Country Name_id'] + raw_data['Category_id'] + raw_data['Data_id']

cols = raw_id.columns.tolist()
cols.insert(0, cols.pop(cols.index('id')))
raw_id = raw_id[cols]

In [33]:
raw_clean = raw_id.drop(raw_id.columns[1:9], axis=1)
raw_clean.columns = raw_clean.columns.str[0:4]
raw_clean = raw_clean.replace("..", np.nan)
raw_clean.to_csv('raw_clean.csv', index=False)

/var/folders/qz/xwx1r5sx2k9dgf6nxmgph3200000gn/T/ipykernel_29283/204907578.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_clean = raw_clean.replace("..", np.nan)


In [34]:
raw_clean.head()

,id,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,0010105000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.888577e+10,1.857257e+10,1.682600e+10,1.775189e+10,1.714322e+10,1.773194e+10,1.858582e+10,1.362107e+10,NaN,NaN
1,0020105000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.759168e+02,5.489685e+02,4.848906e+02,4.974060e+02,4.665706e+02,4.684035e+02,4.757181e+02,3.405232e+02,NaN,NaN
2,0030105000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0040105000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0050104000005,1.878951e+07,1.889035e+07,1.895606e+07,1.878242e+07,1.821171e+07,1.780392e+07,1.437832e+07,1.410682e+07,1.416699e+07,...,1.643133e+08,2.068983e+08,1.903312e+08,1.788537e+08,2.378150e+08,2.479358e+08,2.323007e+08,1.822300e+08,NaN,NaN
